In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **<font color='8d5383'>Import library</font>**

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

# **<font color='8d5383'>Import Dataset</font>**

In [ ]:


try:
    # Load the biochemists dataset from the new path
    df = pd.read_csv('/content/drive/MyDrive/ttn/bioChemists.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: The specified file was not found. Please ensure the file is in the correct path.")
    exit()

# Drop the rownames column as it's not a feature
df = df.drop(columns=['rownames'])

# Convert categorical variables to numerical using LabelEncoder
categorical_cols = ['fem', 'mar']
for column in categorical_cols:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

# Define features (X) and target variable (y)
X = df.drop(columns=['art'])
y = df['art']

# Normalize numerical columns for better model performance
numerical_cols = ['kid5', 'phd', 'ment']
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# **<font color='8d5383'>Modeling and Training</font>**

In [ ]:

#  Define Custom Loss Functions


def poisson_loss(y_true, y_pred_log_lambda):
    """Calculates Poisson negative log-likelihood loss."""
    y_pred_lambda = torch.exp(y_pred_log_lambda)
    loss = y_pred_lambda - y_true * y_pred_log_lambda
    return torch.mean(loss)

def cmp_loss(y_true, y_pred):
    """Calculates CMP negative log-likelihood loss."""
    y_pred_log_lambda = y_pred[:, 0].unsqueeze(1)
    y_pred_log_nu = y_pred[:, 1].unsqueeze(1)

    y_pred_nu = torch.exp(y_pred_log_nu)

    # Use lgamma for log(y!)
    log_y_true_factorial = torch.lgamma(y_true + 1.0)

    # Numerically approximate the normalization constant Z
    s = torch.arange(500, dtype=torch.float32, device=y_true.device).unsqueeze(0)

    log_z_term = s * y_pred_log_lambda - y_pred_nu * torch.lgamma(s + 1.0)
    log_Z = torch.logsumexp(log_z_term, dim=1, keepdim=True)

    # Calculate CMP loss
    loss = -(y_true * y_pred_log_lambda - y_pred_nu * log_y_true_factorial - log_Z)
    return torch.mean(loss)

def zicomp_loss(y_true, y_pred):
    """Calculates ZICMP negative log-likelihood loss."""
    y_pred_log_lambda = y_pred[:, 0].unsqueeze(1)
    y_pred_log_nu = y_pred[:, 1].unsqueeze(1)
    y_pred_logit_pi = y_pred[:, 2].unsqueeze(1)

    y_pred_pi = torch.sigmoid(y_pred_logit_pi)

    # Numerically approximate Z
    s = torch.arange(500, dtype=torch.float32, device=y_true.device).unsqueeze(0)
    log_z_term = s * y_pred_log_lambda - torch.exp(y_pred_log_nu) * torch.lgamma(s + 1.0)
    log_Z = torch.logsumexp(log_z_term, dim=1, keepdim=True)

    # Create masks for y_true == 0 and y_true > 0
    is_zero = (y_true == 0).float()
    is_not_zero = (y_true > 0).float()

    # Calculate loss for y_true = 0
    log_prob_zero = torch.log(y_pred_pi + (1 - y_pred_pi) * torch.exp(-log_Z))
    loss_zero = -is_zero * log_prob_zero

    # Calculate loss for y_true > 0
    log_y_true_factorial = torch.lgamma(y_true + 1.0)
    log_prob_nonzero = torch.log(1 - y_pred_pi) + y_true * y_pred_log_lambda - torch.exp(y_pred_log_nu) * log_y_true_factorial - log_Z
    loss_nonzero = -is_not_zero * log_prob_nonzero

    total_loss = loss_zero + loss_nonzero
    return torch.mean(total_loss)

#  Build and Train Models


# Base Neural Network Model
class CountModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CountModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2), # Add dropout for regularization
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.network(x)

def train_model(model, loss_function, epochs, X_train, y_train, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_function(y_train, outputs)
        loss.backward()
        optimizer.step()
    return model

# Poisson Model
print("\nTraining Poisson model...")
poisson_model = CountModel(X_train.shape[1], 1)
poisson_model = train_model(poisson_model, poisson_loss, epochs=200, X_train=X_train_tensor, y_train=y_train_tensor)
poisson_preds = torch.exp(poisson_model(X_test_tensor)).detach().numpy()
poisson_mae = mean_absolute_error(y_test_tensor.numpy(), poisson_preds)
poisson_mse = mean_squared_error(y_test_tensor.numpy(), poisson_preds)
poisson_rmse = np.sqrt(poisson_mse)
print("Poisson model trained.")

# CMP Model
print("\nTraining CMP model...")
cmp_model = CountModel(X_train.shape[1], 2)
cmp_model = train_model(cmp_model, cmp_loss, epochs=200, X_train=X_train_tensor, y_train=y_train_tensor)
cmp_preds = torch.exp(cmp_model(X_test_tensor)[:, 0].unsqueeze(1)).detach().numpy()
cmp_mae = mean_absolute_error(y_test_tensor.numpy(), cmp_preds)
cmp_mse = mean_squared_error(y_test_tensor.numpy(), cmp_preds)
cmp_rmse = np.sqrt(cmp_mse)
print("CMP model trained.")

# ZICMP Model - Reduced Learning Rate for Better Convergence
print("\nTraining ZICMP model...")
zicomp_model = CountModel(X_train.shape[1], 3)
# Use a smaller learning rate for this complex model to prevent divergence
zicomp_model = train_model(zicomp_model, zicomp_loss, epochs=200, X_train=X_train_tensor, y_train=y_train_tensor, lr=0.005)
zicomp_preds = torch.exp(zicomp_model(X_test_tensor)[:, 0].unsqueeze(1)).detach().numpy()
zicomp_mae = mean_absolute_error(y_test_tensor.numpy(), zicomp_preds)
zicomp_mse = mean_squared_error(y_test_tensor.numpy(), zicomp_preds)
zicomp_rmse = np.sqrt(zicomp_mse)
print("ZICMP model trained.")

# **<font color='8d5383'>Evaluation</font>**

In [ ]:

# 4. Compare Performance


print("\n--- Model Comparison Results ---")
print("Poisson Model:")
print(f"  MAE: {poisson_mae:.4f}")
print(f"  MSE: {poisson_mse:.4f}")
print(f"  RMSE: {poisson_rmse:.4f}")

print("\nCMP Model:")
print(f"  MAE: {cmp_mae:.4f}")
print(f"  MSE: {cmp_mse:.4f}")
print(f"  RMSE: {cmp_rmse:.4f}")

print("\nZICMP Model:")
print(f"  MAE: {zicomp_mae:.4f}")
print(f"  MSE: {zicomp_mse:.4f}")
print(f"  RMSE: {zicomp_rmse:.4f}")

# Display data distribution and zeros
print("\n--- Target Variable Distribution (art) ---")
print(df['art'].value_counts().sort_index())
print(f"\nPercentage of zeros in the data: {np.mean(df['art'] == 0) * 100:.2f}%")


Data loaded successfully.

Training Poisson model...
Poisson model trained.

Training CMP model...
CMP model trained.

Training ZICMP model...
ZICMP model trained.

--- Model Comparison Results ---
Poisson Model:
  MAE: 1.5400
  MSE: 5.5249
  RMSE: 2.3505

CMP Model:
  MAE: 1.4427
  MSE: 6.0218
  RMSE: 2.4539

ZICMP Model:
  MAE: 1.4334
  MSE: 5.1225
  RMSE: 2.2633

--- Target Variable Distribution (art) ---
art
0     275
1     246
2     178
3      84
4      67
5      27
6      17
7      12
8       1
9       2
10      1
11      1
12      2
16      1
19      1
Name: count, dtype: int64

Percentage of zeros in the data: 30.05%
